In [42]:
import pandas as pd
import os
os.chdir("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers")
mypath = os.getcwd()

# table date 


In [43]:
df = pd.read_csv(str(mypath)+"/df_clean/df_clean.csv",sep=';')
df.columns

Index(['Names', 'Country', 'room_type', 'nuitee', 'reservation_date',
       'traveler_infos', 'date_review', 'review_title', 'grade_review',
       'positive_review', 'negative_review', 'usefulness_review', 'UniqueID',
       'hotel', 'level_grade_review', 'level_hotel', 'month_str', 'month_num',
       'year', 'delay_comment', 'date'],
      dtype='object')

In [44]:
df["id_date"] = df["date"].astype(str)
df["id_date"] = df["id_date"].str.replace("-","")

In [45]:
champs_date = ["month_str","month_num","year","date", "id_date"]

In [46]:
df_date = df[champs_date].copy()
df_date.drop_duplicates(keep='first', inplace=True)
df_date.reset_index(drop=True, inplace=True)
df_date.to_csv(str(mypath)+"/tables/date.csv",sep=';', index=False, encoding="utf-8-sig")

# Client

In [47]:
champs_client = ["Country", "nuitee", "traveler_infos", "review_title", "positive_review", "negative_review", "usefulness_review", "delay_comment"]

In [48]:
df_client = df[champs_client].copy()
df_client.rename(columns={'Country': 'country'}, inplace=True)
import uuid
df_client['id_client'] = df_client.apply(lambda _: uuid.uuid4(), axis=1)
df_client.to_csv(str(mypath)+"/tables/client.csv",sep=';', index=False, encoding="utf-8-sig")

# hotel

In [49]:
champs_hotel = ["hotel", "level_hotel"]

In [50]:
df_hotel = df[champs_hotel].copy()
df_hotel.drop_duplicates(keep='first', inplace=True)
df_hotel.dropna(inplace=True)
import uuid
df_hotel['id_hotel'] = df_hotel.apply(lambda _: uuid.uuid4(), axis=1)
df_hotel.reset_index(drop=True, inplace=True)
df_hotel.to_csv(str(mypath)+"/tables/hotel.csv",sep=';', index=False, encoding="utf-8-sig")

# chambre

In [51]:
champs_chambre = ["room_type", "hotel"]

In [52]:
df_room = df[champs_chambre].copy()
df_room.drop_duplicates(keep='first', inplace=True)
import uuid
df_room.reset_index(drop=True, inplace=True)

#merge pour avoir id_hotel
df_room=df_room.merge(df_hotel, on='hotel')
df_room['id_room'] = df_room.apply(lambda _: uuid.uuid4(), axis=1)
df_room.drop(columns=['hotel', 'level_hotel'], inplace=True)
df_room.to_csv(str(mypath)+"/tables/room.csv",sep=';', index=False, encoding="utf-8-sig")

In [53]:
df_room

,room_type,id_hotel,id_room
0,Chambre Standard Woody's Roundup,8e91af37-81a4-40d6-a123-708455813b70,3fd69528-3808-42db-8b85-5478b1092a14
1,NaN,8e91af37-81a4-40d6-a123-708455813b70,cb6b015a-de66-44ac-8039-a151f9760e81
2,Chambre Double/Lit Gigogne Woody's Roundup - C...,8e91af37-81a4-40d6-a123-708455813b70,ea86ade4-acf9-45f9-960d-5136838c2649
3,Chambre Woody's Roundup à Proximité des Instal...,8e91af37-81a4-40d6-a123-708455813b70,368c41a7-0b4c-416e-930f-c1da19b32170
4,Chambre Standard Woody's Roundup (2 Adultes + ...,8e91af37-81a4-40d6-a123-708455813b70,01c8911e-7298-44f8-b630-a6e155ad5e96
...,...,...,...
93,Chambre Golden Forest Club – Côté Lac (3 Adult...,5737b66b-b96a-49eb-94e2-5c144f4e3334,e05b3ac3-5ee6-45a2-bbd2-e7bff8b413b7
94,Chambre Standard proche Installations Hôtelièr...,5737b66b-b96a-49eb-94e2-5c144f4e3334,82fd3421-4cd7-4a54-a784-f706112b55ca
95,Chambre Golden Forest Club (2 Adultes + 2 Enfa...,5737b66b-b96a-49eb-94e2-5c144f4e3334,1044ce77-eb83-41b2-a960-c4a47bc8e875
96,Chambre Golden Forest Club (3 Adultes + 1 Enfant),5737b66b-b96a-49eb-94e2-5c144f4e3334,3051bb40-53ee-49a0-8ef4-e30e461d44d6


# Reservation

In [54]:
champs_res=["grade_review", "level_grade_review", "id_date", "room_type"]

In [55]:
df_res = df[champs_res].copy()
import uuid
df_res.reset_index(drop=True, inplace=True)

#merge pour avoir tous les id
#doit avoir 11219 lignes
df_res['id_reservation'] = df_res.apply(lambda _: uuid.uuid4(), axis=1)
df_res = pd.merge(df_res, df_room[["id_room", "room_type"]], how="left", on = ["room_type"])
df_res.drop_duplicates(subset=['id_reservation'], inplace=True)
df_res.reset_index(drop=True, inplace=True)

df_res.drop(columns=['room_type'], inplace=True)
df_res = pd.concat([df_res, df_client['id_client']], axis=1) 
df_res.to_csv(str(mypath)+"/tables/reservation.csv",sep=';', index=False, encoding="utf-8-sig")